# Unsupervised Lab Session

## Learning outcomes:
- Exploratory data analysis and data preparation for model building.
- PCA for dimensionality reduction.
- K-means and Agglomerative Clustering

## Problem Statement
Based on the given marketing campigan dataset, segment the similar customers into suitable clusters. Analyze the clusters and provide your insights to help the organization promote their business.

## Context:
- Customer Personality Analysis is a detailed analysis of a company’s ideal customers. It helps a business to better understand its customers and makes it easier for them to modify products according to the specific needs, behaviors and concerns of different types of customers.
- Customer personality analysis helps a business to modify its product based on its target customers from different types of customer segments. For example, instead of spending money to market a new product to every customer in the company’s database, a company can analyze which customer segment is most likely to buy the product and then market the product only on that particular segment.

## About dataset
- Source: https://www.kaggle.com/datasets/imakash3011/customer-personality-analysis?datasetId=1546318&sortBy=voteCount

### Attribute Information:
- ID: Customer's unique identifier
- Year_Birth: Customer's birth year
- Education: Customer's education level
- Marital_Status: Customer's marital status
- Income: Customer's yearly household income
- Kidhome: Number of children in customer's household
- Teenhome: Number of teenagers in customer's household
- Dt_Customer: Date of customer's enrollment with the company
- Recency: Number of days since customer's last purchase
- Complain: 1 if the customer complained in the last 2 years, 0 otherwise
- MntWines: Amount spent on wine in last 2 years
- MntFruits: Amount spent on fruits in last 2 years
- MntMeatProducts: Amount spent on meat in last 2 years
- MntFishProducts: Amount spent on fish in last 2 years
- MntSweetProducts: Amount spent on sweets in last 2 years
- MntGoldProds: Amount spent on gold in last 2 years
- NumDealsPurchases: Number of purchases made with a discount
- AcceptedCmp1: 1 if customer accepted the offer in the 1st campaign, 0 otherwise
- AcceptedCmp2: 1 if customer accepted the offer in the 2nd campaign, 0 otherwise
- AcceptedCmp3: 1 if customer accepted the offer in the 3rd campaign, 0 otherwise
- AcceptedCmp4: 1 if customer accepted the offer in the 4th campaign, 0 otherwise
- AcceptedCmp5: 1 if customer accepted the offer in the 5th campaign, 0 otherwise
- Response: 1 if customer accepted the offer in the last campaign, 0 otherwise
- NumWebPurchases: Number of purchases made through the company’s website
- NumCatalogPurchases: Number of purchases made using a catalogue
- NumStorePurchases: Number of purchases made directly in stores
- NumWebVisitsMonth: Number of visits to company’s website in the last month

### 1. Import required libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans, AgglomerativeClustering
from sklearn.metrics import silhouette_score
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

### 2. Load the CSV file (i.e marketing.csv) and display the first 5 rows of the dataframe. Check the shape and info of the dataset.

In [ ]:
df = pd.read_csv('marketing.csv')
print(df.head())
print(df.shape)
print(df.info())


### 3. Check the percentage of missing values? If there is presence of missing values, treat them accordingly.

In [ ]:
missing_percentage = df.isnull().mean() * 100
print(missing_percentage)


### 4. Check if there are any duplicate records in the dataset? If any drop them.

In [ ]:
df = df.drop_duplicates()


### 5. Drop the columns which you think redundant for the analysis 

In [ ]:
columns_to_drop = ['ID', 'Dt_Customer']
df = df.drop(columns=columns_to_drop)


### 6. Check the unique categories in the column 'Marital_Status'
- i) Group categories 'Married', 'Together' as 'relationship'
- ii) Group categories 'Divorced', 'Widow', 'Alone', 'YOLO', and 'Absurd' as 'Single'.

In [ ]:
df['Marital_Status'] = df['Marital_Status'].replace({'Married': 'relationship', 'Together': 'relationship'})
df['Marital_Status'] = df['Marital_Status'].replace({'Divorced': 'Single', 'Widow': 'Single', 'Alone': 'Single', 'YOLO': 'Single', 'Absurd': 'Single'})


### 7. Group the columns 'MntWines', 'MntFruits', 'MntMeatProducts', 'MntFishProducts', 'MntSweetProducts', and 'MntGoldProds' as 'Total_Expenses'

In [ ]:
df['Total_Expenses'] = df[['MntWines', 'MntFruits', 'MntMeatProducts', 'MntFishProducts', 'MntSweetProducts', 'MntGoldProds']].sum(axis=1)


### 8. Group the columns 'NumWebPurchases', 'NumCatalogPurchases', 'NumStorePurchases', and 'NumDealsPurchases' as 'Num_Total_Purchases'

In [ ]:
df['Num_Total_Purchases'] = df[['NumWebPurchases', 'NumCatalogPurchases', 'NumStorePurchases', 'NumDealsPurchases']].sum(axis=1)


### 9. Group the columns 'Kidhome' and 'Teenhome' as 'Kids'

In [ ]:
df['Kids'] = df['Kidhome'] + df['Teenhome']


### 10. Group columns 'AcceptedCmp1 , 2 , 3 , 4, 5' and 'Response' as 'TotalAcceptedCmp'

In [ ]:
df['TotalAcceptedCmp'] = df[['AcceptedCmp1', 'AcceptedCmp2', 'AcceptedCmp3', 'AcceptedCmp4', 'AcceptedCmp5', 'Response']].sum(axis=1)


### 11. Drop those columns which we have used above for obtaining new features

In [ ]:
columns_to_drop = ['MntWines', 'MntFruits', 'MntMeatProducts', 'MntFishProducts', 'MntSweetProducts', 'MntGoldProds',
                   'NumWebPurchases', 'NumCatalogPurchases', 'NumStorePurchases', 'NumDealsPurchases',
                   'AcceptedCmp1', 'AcceptedCmp2', 'AcceptedCmp3', 'AcceptedCmp4', 'AcceptedCmp5', 'Response',
                   'Kidhome', 'Teenhome']
df = df.drop(columns=columns_to_drop)


### 12. Extract 'age' using the column 'Year_Birth' and then drop the column 'Year_birth'

In [ ]:
df['Age'] = 2023 - df['Year_Birth']
df = df.drop(columns=['Year_Birth'])


### 13. Encode the categorical variables in the dataset

In [ ]:
df = pd.get_dummies(df, drop_first=True)


### 14. Standardize the columns, so that values are in a particular range

In [ ]:
scaler = StandardScaler()
df_scaled = scaler.fit_transform(df)


### 15. Apply PCA on the above dataset and determine the number of PCA components to be used so that 90-95% of the variance in data is explained by the same.

In [ ]:
pca = PCA()
pca.fit(df_scaled)
explained_variance_ratio = np.cumsum(pca.explained_variance_ratio_)
num_components_90 = np.argmax(explained_variance_ratio >= 0.90) + 1
num_components_95 = np.argmax(explained_variance_ratio >= 0.95) + 1


### 16. Apply K-means clustering and segment the data (Use PCA transformed data for clustering)

In [ ]:
kmeans = KMeans(n_clusters=5, random_state=42)
df['Cluster_KMeans'] = kmeans.fit_predict(df_scaled)


### 17. Apply Agglomerative clustering and segment the data (Use Original data for clustering), and perform cluster analysis by doing bivariate analysis between the cluster label and different features and write your observations.

In [ ]:
agg_clustering = AgglomerativeClustering(n_clusters=5)
df['Cluster_Agg'] = agg_clustering.fit_predict(df)

# Bivariate analysis between the cluster label and different features
plt.figure(figsize=(12, 8))
sns.countplot(x='Cluster_Agg', hue='relationship_Single', data=df)
plt.title('Cluster Analysis: Relationship Status')
plt.show()

plt.figure(figsize=(12, 8))
sns.countplot(x='Cluster_Agg', hue='TotalAcceptedCmp', data=df)
plt.title('Cluster Analysis: Total Accepted Campaigns')
plt.show()

### Visualization and Interpretation of results

In [ ]:
plt.figure(figsize=(12, 8))
sns.countplot(x='Cluster_Agg', data=df)
plt.title('Distribution of Clusters')
plt.show()

-----
## Happy Learning
-----